In [1]:
from langgraph.graph import StateGraph,START, END
from typing import TypedDict, Literal, Annotated
from langchain_openai import ChatOpenAI
from langchain_core.messages import SystemMessage, HumanMessage,BaseMessage
from langgraph.checkpoint.memory import InMemorySaver
from dotenv import load_dotenv
import uuid

load_dotenv()
uuid = str(uuid.uuid4())
checkpointer = InMemorySaver()
llm = ChatOpenAI()

/Users/priyajhanji/Documents/personal-git-abhikaura25/ai-engineering-hub/venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


In [2]:

class JokeState(TypedDict):
    topic: str
    joke: str
    explanation: str

def generate_joke(state: JokeState) -> JokeState:
    prompt = f"Tell me a joke about {state['topic']}."
    joke = llm.invoke([HumanMessage(content=prompt)])
    state['joke'] = joke.content
    return state

def explain_joke(state: JokeState) -> JokeState:
    prompt = f"Explain the following joke: {state['joke']}"
    explanation = llm.invoke([HumanMessage(content=prompt)])
    state['explanation'] = explanation.content
    return state


In [3]:

graph = StateGraph(JokeState)

## add nodes
graph.add_node("generate_joke", generate_joke)
graph.add_node("explain_joke", explain_joke)
graph.add_edge(START, "generate_joke")
graph.add_edge("generate_joke", "explain_joke")
graph.add_edge("explain_joke", END)
workflow = graph.compile(checkpointer=checkpointer)
config = {"configurable": {"thread_id": uuid}}

result = workflow.invoke({"topic": "pizza"}, config=config)

print("Joke:", result['joke'])
print("Explanation:", result['explanation'])


Joke: Why did the pizza go to the psychiatrist?
Because it felt like its life was in pieces!
Explanation: This joke plays on the idea of a pizza "feeling" like its life is in pieces, which is a pun on the literal meaning of the word "pieces" (referring to slices of pizza) and the figurative meaning of feeling emotionally distressed or fragmented. It humorously suggests that the pizza is seeking therapy because it is feeling "broken" or overwhelmed, just like a person might feel when they go to see a psychiatrist.


In [8]:
list(workflow.get_state_history(config))

[StateSnapshot(values={'topic': 'programming'}, next=('generate_joke',), config={'configurable': {'thread_id': 'ecbce0e5-70b3-45e5-a89a-86175e0c51fb', 'checkpoint_ns': '', 'checkpoint_id': '1f0fede8-bb07-6952-8001-ac811d728bd8'}}, metadata={'source': 'update', 'step': 1, 'parents': {}}, created_at='2026-01-31T19:53:42.827854+00:00', parent_config={'configurable': {'thread_id': 'ecbce0e5-70b3-45e5-a89a-86175e0c51fb', 'checkpoint_ns': '', 'checkpoint_id': '1f0fedd5-a063-614a-8000-1bd8ea2b38f9'}}, tasks=(PregelTask(id='794b71ad-b423-e0ee-de24-cfd7a015e20e', name='generate_joke', path=('__pregel_pull', 'generate_joke'), error=None, interrupts=(), state=None, result=None),), interrupts=()),
 StateSnapshot(values={'topic': 'pizza', 'joke': 'Why did the pizza go to the psychiatrist?\nBecause it felt like its life was in pieces!', 'explanation': 'This joke plays on the idea of a pizza "feeling" like its life is in pieces, which is a pun on the literal meaning of the word "pieces" (referring to

In [7]:
workflow.update_state(
    config={"configurable": {"thread_id": uuid, "checkpoint_ns": "", "checkpoint_id": "1f0fedd5-a063-614a-8000-1bd8ea2b38f9"}},
    values={"topic": "programming"}
)

{'configurable': {'thread_id': 'ecbce0e5-70b3-45e5-a89a-86175e0c51fb',
  'checkpoint_ns': '',
  'checkpoint_id': '1f0fede8-bb07-6952-8001-ac811d728bd8'}}

In [10]:
workflow.invoke(None, config={"configurable": {"thread_id": uuid, "checkpoint_ns": "", "checkpoint_id": "1f0fede8-bb07-6952-8001-ac811d728bd8"}})

{'topic': 'programming',
 'joke': 'Why do programmers prefer dark mode?\n\nBecause light attracts bugs!',
 'explanation': 'This joke is a play on words, as "bugs" can refer to both insects and errors in computer programming. In this case, it suggests that programmers prefer working in dark mode on their computers because, just like light attracts insects (bugs), light mode on the computer screen can attract errors (bugs) in their code. So, by working in dark mode, programmers may be able to minimize the number of errors they encounter while programming.'}